<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 

## 03 - DBSCAN ##

**Table of Contents**
<br>
This notebook uses GPU-accelerated DBSCAN to identify clusters of infected people. This notebook covers the below sections: 
1. [Environment](#Environment)
2. [Load Data](#Load-Data)
3. [DBSCAN Clustering](#DBSCAN-Clustering)
    * [Exercise #1 - Make Another DBSCAN Instance](#Exercise-#1---Make-Another-DBSCAN-Instance)
4. [Visualize the Clusters](#Visualize-the-Clusters)

## Environment ##

In [ ]:
import cudf
import cuml

import cuxfilter as cxf

## Load Data ##
For this notebook, we again load a subset of our population data with only the columns we need. An `infected` column has been added to the data to indicate whether or not a person is known to be infected with our simulated virus.

In [ ]:
gdf = cudf.read_csv('./data/pop_sample.csv', dtype=['float32', 'float32', 'float32'])
print(gdf.dtypes)
gdf.shape

In [ ]:
gdf.head()

In [ ]:
gdf['infected'].value_counts()

## DBSCAN Clustering ##
DBSCAN is another unsupervised clustering algorithm that is particularly effective when the number of clusters is not known up front and the clusters may have concave or other unusual shapes--a situation that often applies in geospatial analytics.

In this series of exercises you will use DBSCAN to identify clusters of infected people by location, which may help us identify groups becoming infected from common patient zeroes and assist in response planning.

Create a DBSCAN instance by using `cuml.DBSCAN`. Pass in the named argument `eps` (the maximum distance a point can be from the nearest point in a cluster to be considered possibly in that cluster) to be `5000`. Since the `northing` and `easting` values we created are measured in meters, this will allow us to identify clusters of infected people where individuals may be separated from the rest of the cluster by up to 5 kilometers.

Below we train a DBSCAN algorithm. We start by creating a new dataframe from rows of the original dataframe where `infected` is `1` (true), and call it `infected_df`--be sure to reset the dataframe's index afterward. Use `dbscan.fit_predict` to perform clustering on the `northing` and `easting` columns of `infected_df`, and turn the resulting series into a new column in `infected_gdf` called "cluster". Finally, compute the number of clusters identified by DBSCAN.

In [ ]:
dbscan = cuml.DBSCAN(eps=5000)
# dbscan = cuml.DBSCAN(eps=10000)

infected_df = gdf[gdf['infected'] == 1].reset_index()
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()

### Exercise #1 - Make Another DBSCAN Instance ###

**Instructions**: <br>
* Modify the `<FIXME>` only and execute the below cell to instantiate a DBSCAN instance with `10000` for `eps`.
* Modify the `<FIXME>` only and execute the cell below to fit the data and identify infected clusters. 

In [ ]:
dbscan = cuml.DBSCAN(<<<<FIXME>>>>)

In [ ]:
infected_df = gdf[gdf['infected'] == 1].reset_index()
infected_df['cluster'] = dbscan.<<<<FIXME>>>>(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()

Click ... for solution. 

## Visualize the Clusters

Because we have the same column names as in the K-means example--`easting`, `northing`, and `cluster`--we can use the same code to visualize the clusters.

In [ ]:
infected_df.to_pandas().plot(kind='scatter', x='easting', y='northing', c='cluster')

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** Let's move to the [next notebook](3-04_logistic_regression.ipynb). 

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">